<H1><CENTER>OPI PROCING</CENTER></H1>
<img src='./Data/Capture.JPG'>
<H3><CENTER>Descripcion:</CENTER></H3>
<H5><CENTER>Crear clusters de los Hoteles para determinar en cuales se puede mover la tarifa.</CENTER></H5>

### IMPORTACION DE LIBRERIAS

In [183]:
###
# IMPORTACIONES NECESARIAS
###
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import scipy as sp
import plotly 
from IPython.display import IFrame 
import matplotlib.pyplot as plt
import cufflinks as cf
import statistics

In [63]:
%matplotlib inline

In [2]:
###
# API KEY PARA PLOTLY
###
plotly.tools.set_credentials_file(username='Israel.Vargas', api_key='wqx2U9kneSEyGJ4ZjrsY')

### LIMPIEZA DE LOS DATOS:
**Cargamos los datos, rellenamos los datos del canal de posadas que sean nulos con "WALKIN" y creamos una agrupacion con una nueva columna que contenga la frecuencia de los dias de diferencia entre la fecha de llegada y la fecha de reserva**

In [6]:
###
# Cargando Data
###
data = pd.read_csv(r"./Data/Denograma.csv",low_memory=False)
data.head()

,Hotel,CANAL POSADAS,Marca,num_Resv,days
0,OAC,NaN,ONE,1,NaN
1,OAC,NaN,ONE,1,0.0
2,OAC,NaN,ONE,1,0.0
3,OAC,NaN,ONE,1,0.0
4,OAC,NaN,ONE,2,0.0


In [7]:
###
# Rellenando los WALKIN
###
data.fillna(value={'CANAL POSADAS':'WALKIN'},inplace=True)
data.head()

In [23]:
###
# Agrupando por Canal y Hotel la frecuencia de los días 
###
freq = data.groupby(['Hotel','CANAL POSADAS'])['days'].value_counts().reset_index(name="Freq")
print(freq.shape)
freq.head(30)

(9445, 4)


,Hotel,CANAL POSADAS,days,Freq
0,OAC,CORPO-RATE,0.0,35
1,OAC,CORPO-RATE,1.0,23
2,OAC,CORPO-RATE,4.0,18
3,OAC,CORPO-RATE,3.0,17
4,OAC,CORPO-RATE,2.0,14
5,OAC,CORPO-RATE,5.0,10
6,OAC,CORPO-RATE,6.0,8
7,OAC,CORPO-RATE,7.0,8
8,OAC,CORPO-RATE,8.0,8
9,OAC,CORPO-RATE,10.0,4


### VISUALIZACIÓN:

**Mostramos el histograma de la frecuencia para comprender hacia donde se carga la información de los dias.**

In [180]:
###
# PREPARANDO GRAFICACIÓN
###
data_plotly = [go.Histogram(y=freq.days)]

In [181]:
###
# CREANDO HISTOGRAMA DE LA FREQUENCIA
###
py.iplot(data_plotly)

##### Observamos que la información se carga más a los días que son de -n hasta 14, siendo este rango el más alto en frecuencia. Por tal motivo quitamos los WALKIN y procedemos a realizar un análisis más profundo de los datos.

In [52]:
#w_W = widouth_wolkins
w_w = freq.loc[freq['CANAL POSADAS'] != 'WALKIN']
w_w.head()

,Hotel,CANAL POSADAS,days,Freq
0,OAC,CORPO-RATE,0.0,35
1,OAC,CORPO-RATE,1.0,23
2,OAC,CORPO-RATE,4.0,18
3,OAC,CORPO-RATE,3.0,17
4,OAC,CORPO-RATE,2.0,14


### AGURPACIÓN POR LA FRECUENCIA MÁXIMA.
**¿Porque la frecuencia máxima? <br> La razón e smuy simple, no tiene ningun caso observar la frecuencia minima ya que la intención es poder tener un tiempo adecuado para que OPI pueda mover los precios, si a OPI se le da un rango de tiempo adecuado, se puede ajustar la tarifa de cada hotel con anticipación y mejorar los ingresos de la compañia.**

In [156]:
aux = w_w.groupby(['Hotel','CANAL POSADAS'])['days'].max().reset_index(name="Max_Val")
aux.head(10)

,Hotel,CANAL POSADAS,Max_Val
0,OAC,CORPO-RATE,201.0
1,OAC,GDS,33.0
2,OAC,GRUPOS RL,151.0
3,OAC,HOTEL DATOS,72.0
4,OAC,HOTEL INTEGRACION,32.0
5,OAC,OTAs,310.0
6,OAC,VOZ,148.0
7,OAC,WEB,211.0
8,OAG,CORPO-RATE,23.0
9,OAG,GDS,25.0


In [107]:
aux.loc[aux['Hotel'] == hotel]

,Hotel,CANAL POSADAS,Max_Val
0,OAC,CORPO-RATE,201.0
1,OAC,GDS,33.0
2,OAC,GRUPOS RL,151.0
3,OAC,HOTEL DATOS,72.0
4,OAC,HOTEL INTEGRACION,32.0
5,OAC,OTAs,310.0
6,OAC,VOZ,148.0
7,OAC,WEB,211.0


In [146]:
list_h = aux['Hotel'].unique()
print(len(list_h))
list_h

44


array(['OAC', 'OAG', 'OAL', 'OAX', 'OCC', 'OCF', 'OCL', 'OCN', 'OCZ',
       'ODR', 'OGA', 'OGC', 'OGD', 'OGS', 'OGV', 'OGX', 'OIR', 'OLA',
       'OLP', 'OMN', 'OMT', 'OMX', 'OPC', 'OPF', 'OPJ', 'OPS', 'OPT',
       'OPV', 'OPZ', 'OQA', 'OQR', 'OQS', 'ORY', 'OSC', 'OSD', 'OSL',
       'OSM', 'OSP', 'OTL', 'OVA', 'OVC', 'OVM', 'OXC', 'OXL'],
      dtype=object)

## VIASUALIZACIÓN DE LA FRECUENCIA MÁS ALTA

### <CENTER>GRAFICA QUE MUESTRA LA FRECUENCIA MÁS ALTA POR CANAL DE RESERVACIÓN DE CADA HOTEL</CENTER>

In [173]:
###
# PLOTENADO EL MÁXIMO DE DIAS DE RESERVA POR HOTEL
###
canales = ['CORPO-RATE', 'GDS', 'GRUPOS RL','HOTEL DATOS','HOTEL INTEGRACION','OTAs','VOZ','WEB']
data_aux = []
for hotel in list_h:
    m_v = aux.loc[aux['Hotel'] == hotel]
    data_aux.append(go.Bar(name=hotel,x=canales,y=m_v['Max_Val']))

fig = go.Figure(data=data_aux)
fig.layout.update(barmode='group')
py.iplot(fig)

### <CENTER>GRAFICA QUE MUESTRA POR HOTEL LA FRECUENCIA MÁS ALTA EN CADA UNO DE SUS CANALES</CENTER>

In [123]:
###
# PLOTENADO EL MÁXIMO VALOR DE CADA CANAL POR HOTEL
###
canales = ['CORPO-RATE', 'GDS', 'GRUPOS RL','HOTEL DATOS','HOTEL INTEGRACION','OTAs','VOZ','WEB']
hoteles = ['OAC', 'OAG', 'OAL', 'OAX', 'OCC', 'OCF', 'OCL', 'OCN', 'OCZ',
       'ODR', 'OGA', 'OGC', 'OGD', 'OGS', 'OGV', 'OGX', 'OIR', 'OLA',
       'OLP', 'OMN', 'OMT', 'OMX', 'OPC', 'OPF', 'OPJ', 'OPS', 'OPT',
       'OPV', 'OPZ', 'OQA', 'OQR', 'OQS', 'ORY', 'OSC', 'OSD', 'OSL',
       'OSM', 'OSP', 'OTL', 'OVA', 'OVC', 'OVM', 'OXC', 'OXL']
data_aux = []
for canal in canales:
    c = aux.loc[aux['CANAL POSADAS'] == canal]
    c.sort_values(by=['Hotel','Max_Val'], ascending=False)
    data_aux.append(go.Bar(name=canal,x=hoteles,y=c['Max_Val']))

fig = go.Figure(data=data_aux)
fig.layout.update(barmode='group')
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL OTAs </CENTER>

In [159]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####OTAs
values = aux.loc[aux['CANAL POSADAS'] == 'OTAs']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='OTAs',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='Median_OTAs',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='Mean_OTAs',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

C:\Users\israel.vargas\AppData\Local\Continuum\anaconda3\envs\EnvPos\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL CORPO-RATE </CENTER>

In [160]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####CORPO-RATE
values = aux.loc[aux['CANAL POSADAS'] == 'CORPO-RATE']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='CORPO-RATE',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='Median_CORPO-RATE',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='Mean_CORPO-RATE',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL GDS </CENTER>

In [167]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####GDS
values = aux.loc[aux['CANAL POSADAS'] == 'GDS']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='GDS',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_GDS',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_GDS',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL GRUPOS RL </CENTER>

In [168]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####GRUPOS RL
values = aux.loc[aux['CANAL POSADAS'] == 'GRUPOS RL']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='GRUPOS RL',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_GRUPOS RL',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_GRUPOS RL',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL HOTEL DATOS </CENTER>

In [169]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####HOTEL DATOS
values = aux.loc[aux['CANAL POSADAS'] == 'HOTEL DATOS']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='HOTEL DATOS',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_HOTEL DATOS',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_HOTEL DATOS',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL HOTEL INTEGRACION </CENTER>

In [170]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####HOTEL INTEGRACION
values = aux.loc[aux['CANAL POSADAS'] == 'HOTEL INTEGRACION']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='HOTEL INTEGRACION',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_HOTEL INTEGRACION',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_HOTEL INTEGRACION',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL VOZ </CENTER>

In [171]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####VOZ
values = aux.loc[aux['CANAL POSADAS'] == 'VOZ']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='VOZ',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_VOZ',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_VOZ',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LA FRECUENCIA MÁS ALTA, LA MEDIA Y EL PROMEDIO POR HOTEL DEL CANAL WEB </CENTER>

In [172]:
###
# EXTRAYENDO POR HOTEL Y CANAL LOS VALORES ORDENADOS. 
###
                        #####WEB
values = aux.loc[aux['CANAL POSADAS'] == 'WEB']
values.sort_values(by=['Max_Val'], ascending=True,inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(name='WEB',x=values['Hotel'].unique(),y=values['Max_Val']))
fig.add_trace(go.Scatter(name='median_WEB',x=values['Hotel'],y=[values['Max_Val'].median()]*44))
fig.add_trace(go.Scatter(name='mean_WEB',x=values['Hotel'],y=[values['Max_Val'].mean()]*44))
py.iplot(fig)

### <CENTER> GRAFICANDO LAS MEDIAS DE CADA CANAL Y CALCULANDO LA MEDIA DE LAS MISMAS JUNTO CON EL VALOR MINIMO </CENTER>
##### Esto con el fin de concluir con que valor nos quedamos para poder hacer la clusterización de los hoteles con base en ese minimo de dias. 

# Conclusión momentanea: Nos quedaremos con el valor minimo de las medias, 12 dias.

In [185]:
y_medians = [129.5,54,34,31,12.5,44,119,90]
x_chanels = ['OTAs','CORPO RATE','GDS','GRUPOS RL','HOTEL DATOS','HOTEL INTEGRACION','VOZ','WEB']
median_of_medians = [statistics.median(y_medians)]*8
min_of_medians = [min(y_medians)]*8
fig = go.Figure()
fig.add_trace(go.Bar(name='Medians',x=x_chanels,y=y_medians))
fig.add_trace(go.Scatter(name='median_of_medians',x=x_chanels,y=median_of_medians))
fig.add_trace(go.Scatter(name='min_of_medians',x=x_chanels,y=min_of_medians))
py.iplot(fig)

# <CENTER>CREANDO CLUSTERS</CENTER>

In [187]:
print(w_w.shape)
print(aux.shape)

(9305, 4)
(365, 3)


In [188]:
aux.head()

,Hotel,CANAL POSADAS,Max_Val
0,OAC,CORPO-RATE,201.0
1,OAC,GDS,33.0
2,OAC,GRUPOS RL,151.0
3,OAC,HOTEL DATOS,72.0
4,OAC,HOTEL INTEGRACION,32.0


In [ ]:
vals = aux.loc[aux['Max_Val'] > 12.5]
y = vals['Max_Val'].min()
trace = go.Scatter(
    x = aux['Hotel'].unique(),
    y = vals,
    text = vals['CANAL POSADAs'].unique(),
    textposition = 'top-center',
    mode = 'markers-text'
)
data = [trace]
py.iplot(data)